In [1]:
import librosa
import librosa.display
import scipy
import numpy as np
from IPython import display

In [2]:
audio_sample_path = "../../Resource/sample/"

audio_noises = {
    "090119_FF_75315_KOMP_info_biling_1_6644289560728637345_1_62.wav": [1424531, 1425657],
    "090119_MF_KOMP_72358_TBC_1_6644427137121067182_1_62.wav": [779530, 781451],
    "090119_MM_KOMP_75260_seting_wifi1_1_6644397802494435751_1_62.wav": [798585, 803913],
    "09_FF_3Jan2019_Telepon_mati_1_6642196769194180990_1_62.wav": [455760, 458270],
    "09_MM_1Jan19_Internet_Lambat_1_6641254934405777725_1_62.wav": [696304, 697742],
    "100119_MF_42144_STB_BIND_FCR_YES_1_6644848937269278905_1_62_01.wav": [2063419, 2069486],
    "11_MM_42874_43656_13_01_2019_Add_on_HD_1_6646001199685378950_1_62.wav": [137071, 141498],
    "17_MF_42544_43266_11022019_Info_Billing_1_6656543982892428498_1_62.wav": [1016580, 1023104],
    "17_MF__42397_43673_9Jan19_LiveTVtidak_muncul_1_6644410459763056919_1_62.wav": [702676, 710408],
    "28_MF_18_12_2018_INFO_PRODUK_1_6636308570894834691_1_62.wav": [1033953, 1037107],
    "8_FF_04-01-2019_info_produk_1_6642623495669883772_1_62.wav": [1086066, 1087886],
    "8_FF_11-02-2019_Cabut_All_1_6656614733888685916_1_62.wav": [1565433, 1566444],
    "8_FF_2-Jan-2019_Komp._ggn_inet_tbc_1_6651814107208031660_1_62.wav": [594124, 594523],
    "8_FF_Telp_Mati_Internet_Tdk_Bisa_Koneksi_USeeTV_Mati_1_6653377024332208084_1_62.wav": [1435774, 1436252],
    "8_FM_04-Feb-19_Info_Billing_1_6653908118513199761_1_62.wav": [158040, 159955],
    "8_FM_08-01-2019_Laporan_Telepon_Berulang_LAPUL.wav": [292727, 294037],
    "8_FM_8-Jan-2019_2P_MATOT_1_6643933920256669236_1_62.wav": [597788, 598979],
    "8_MF_08_01_2019_INFO_PENAMBAHAN_MINIPACK_1_6644049265898367664_1_62.wav": [945382, 951753],
    "8_MM_05_01_2019_Info_Produk___Promo_Indihome_1_6643011232432470173_1_62.wav": [227632, 233511],
    "8_MM_12-01-2019_Cabut_Semua_Produk_1_6645565406533719368_1_62.wav": [735888, 740745],
    "9_FM_21-Jan-2019_Buka_Isolir_BUKIS_1_6648761695425609116_1_62.wav": [1274076, 1281751],
    "9_FM_3-Jan-2019_Tidak_Bisa_Koneksi_FISIK_1_6642223556905221836_1_62.wav": [1377674, 1384560],
    "9_MM_08_01_2019_useetv_eror_panduan_1_6644111207916711359_1_62.wav": [445840, 450482],
    "9_MM_3-Jan-2019_2p_Matot_1_6642309954467343548_1_62.wav": [1418480, 1420426]
}

In [3]:
def get_noise(audio, start_index, end_index, sr=8000):
    return audio[start_index:end_index]

In [4]:
output_path = "../../Testing/analysis_noise/"
noises = []

for filename, noise in audio_noises.items():
    file = audio_sample_path + filename
    out_file = output_path + filename
    audio, sr = librosa.load(file, sr=None)
    clip_noise = get_noise(audio, noise[0], noise[1], sr)
    if len(clip_noise) > 0:
        librosa.output.write_wav(out_file, clip_noise, sr)

##### Mendapatkan noise clip dan noise length

In [5]:
noise_threshold_result = []
noise_length_result = []

for filename, noise in audio_noises.items():
    audio, sr = librosa.load(audio_sample_path + filename, sr=None)
    noise_clip = audio[noise[0]:noise[1]]
    noise_threshold_result.append(np.max(noise_clip))
    noise_length_result.append((noise[1] - noise[0]) / sr)
    
print("Noise Mean: {}".format(sum(noise_threshold_result)/len(noise_threshold_result)))
print("Noise Max: {}".format(max(noise_threshold_result)))
print("Noise Mean Length: {}".format(sum(noise_length_result)/len(noise_length_result)))

Noise Mean: 0.0231781005859375
Noise Max: 0.09814453125
Noise Mean Length: 0.451078125


In [6]:
for noise_max in noise_threshold_result:
    print(noise_max)

0.02331543
0.022460938
0.09814453
0.03753662
0.013946533
0.014434814
0.015411377
0.016784668
0.028961182
0.013580322
0.013092041
0.011291504
0.010101318
0.01361084
0.014251709
0.012237549
0.016326904
0.010467529
0.04711914
0.02218628
0.023071289
0.014160156
0.03338623
0.030395508


In [47]:
def check_if_sample_below_threshold(sample, high_threshold, low_threshold):
    if (sample > 0):
        if (sample > low_threshold):
            if (sample > high_threshold):
                return False
            else:
                return True
        else:
            return False
    else:
        return True

In [191]:
def get_noise(audio_file):
    high_threshold = 0.09814453125
    low_threshold = 0.00006713867
    noise_start_index = -1
    noise_end_index = -1
    noise_length_threshold = 0.3
    
    audio, sr = librosa.load(audio_file, sr=None)
    for i, sample in enumerate(audio):
        if (check_if_sample_below_threshold(sample, high_threshold, low_threshold)):
            if (noise_end_index - noise_start_index == sr * noise_length_threshold):
                return noise_start_index, noise_end_index
            if (noise_start_index == -1):
                noise_start_index = i
            noise_end_index = i
        else:
            noise_start_index = -1
    return 0, 0

In [192]:
a, b = get_noise("../../Resource/sampel_voice_147_polban/09_FF_3Jan2019_Telepon_mati_1_6642196769194180990_1_62.wav")

In [193]:
print(a/8000, b/8000)

114.4145 114.7145


In [157]:
a, b = librosa.load("../../Resource/sampel_voice_147_polban/09_MM_1Jan19_Internet_Lambat_1_6641254934405777725_1_62.wav", sr=None)
print(a[7293:7704])
print(np.max(a[7293:7704]))

[ 3.0517578e-05 -1.2207031e-04 -2.4414062e-04 -3.3569336e-04
 -3.6621094e-04 -3.3569336e-04 -3.0517578e-04 -2.7465820e-04
 -3.3569336e-04 -3.0517578e-04 -2.4414062e-04 -3.3569336e-04
 -3.0517578e-04 -2.1362305e-04 -1.2207031e-04  3.0517578e-05
  1.2207031e-04  2.1362305e-04  2.7465820e-04  3.3569336e-04
  3.0517578e-04  2.1362305e-04  1.2207031e-04 -6.1035156e-05
 -1.5258789e-04 -2.1362305e-04 -2.7465820e-04 -2.4414062e-04
 -2.1362305e-04 -1.8310547e-04 -1.8310547e-04 -9.1552734e-05
  0.0000000e+00  9.1552734e-05  1.8310547e-04  2.4414062e-04
  5.1879883e-04  5.4931641e-04  5.1879883e-04  5.4931641e-04
  4.5776367e-04  3.3569336e-04  2.1362305e-04  3.0517578e-04
  1.8310547e-04 -3.0517578e-05 -3.0517578e-05 -1.5258789e-04
 -2.4414062e-04 -2.4414062e-04 -6.1035156e-05 -9.1552734e-05
 -9.1552734e-05  0.0000000e+00 -2.1362305e-04 -2.7465820e-04
 -1.8310547e-04 -1.8310547e-04 -2.1362305e-04 -1.5258789e-04
 -6.1035156e-05 -6.1035156e-05 -6.1035156e-05 -6.1035156e-05
  3.0517578e-05 -3.05175